In [ ]:
import pandas as pd
import numpy as py
import xgboost as xgb
from sklearn.cross_validation import train_test_split #use to split train-data and test-data

TRAIN_FILE='predict_feactures_lable.csv'
PREDICT_FILE='predict_data.csv'
RESULT="predict_result_logistic.csv"

In [ ]:
#读取数据,输入文件的地址和需要的字段  （因为内存有限，而且有的时候我们没有必要将所有字段都读取）
def get_date(fname):
    #因为数据量较大，一次读入可能会导致内存错误，所以使用pandas的分块chunk进行读取
    reader=pd.read_csv(fname,header=0,iterator=True)#后面的iterator=True是在喊“我要分块读，你不要把我一次读完”
    chunkSize=100000
    chunks=[]
    loop=True
    while loop:
        try:
            chunk=reader.get_chunk(chunkSize)
            chunks.append(chunk)
        except StopIteration:
            loop=False
            print("Iteration is stopped")
    df_ac=pd.concat(chunks,ignore_index=True)
    #df_ac=df_ac.drop_duplicates() #所以原来的程序这里是不对的，应该是所有的都相同才删除吧
    #这个应该也是不要去重的
    return df_ac

In [ ]:
#读取数据,输入文件的地址和需要的字段  （因为内存有限，而且有的时候我们没有必要将所有字段都读取）
def get_date_needlist(fname,rowlist):
    #因为数据量较大，一次读入可能会导致内存错误，所以使用pandas的分块chunk进行读取
    reader=pd.read_csv(fname,header=0,iterator=True)#后面的iterator=True是在喊“我要分块读，你不要把我一次读完”
    chunkSize=100000
    chunks=[]
    loop=True
    while loop:
        try:
            chunk=reader.get_chunk(chunkSize)[rowlist]
            chunks.append(chunk)
        except StopIteration:
            loop=False
            print("Iteration is stopped")
    df_ac=pd.concat(chunks,ignore_index=True)
    return df_ac

In [ ]:
def get_xgboost_model():
    #get train-data and text-date
    X=get_date(TRAIN_FILE)
    X= X.dropna(how = 'any', axis = 0)#如果有缺失数据则删除
    y=X['buy']
    del X['buy']
    #split train-data and text-data
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
   
    #use xgboost
    dtrain=xgb.DMatrix(X_train,label=y_train)
    dtext=xgb.DMatrix(X_test,label=y_test) #it really need different data to test the data
    
    param={'booster':'gbtree','objective':'binary:logistic','eta':0.1.'max_depth':10,'subsample':1.0,/
          'min_child_weight':5,'colsample_bytree':0.2,'scale_pos_weight':0.1,'eval_metric':'auc','gamma':0.2,'lambda':300}
    
    watchlist=[(dtrain,'train'),(dtext,'val')]
    
    print('begain training')
    model=xgb.train(param,dtrain,10,evals=watchlist)#num_boost_round=1
    print('training end')
    
    #save the model
    model.save_model("xgboost001.model")
    
    
      


In [ ]:
def useModelPredict():
        #get predict-data
        X_pre=get_date(PREDICT_FILE) 
        X_pre=X_pre.drop_duplicates(['user_id','sku_id'])
        X_pre = X_pre.dropna(how = 'any', axis = 0)#如果有缺失数据则删除   
        
        #load the model
        bst=xgb.Booser(model_file="xgboost001.model")
        
        #use the model to predict
        y_pre=bst.predict(X_pre)
        result=X_pre([['user_id','sku_id']])
        result['pro']=pd.Serise(y_pre) #buy-probability of every user-product
        
        #结果只需要提交预测为购买的用户，不购买的无需提交
        result=result[result['pro']>0.5]
        #如果一个用户被同时预测会购买多种产品，则只提交最最可能购买的那个产品
        idx=result.groupby(['user_id'])['pro'].transform(max)==result['pro']
        result=result[idx]
        #此时已然会存在一个用户被预测购买多种产品的情形，即概率值相同被同时保存，智能保留一个user_id
        result.drop_duplicates(['user_id'],inplace=True)
        #使user全部是user表中的（本来就是，不用管），product全部是product表中的
        df_pro=get_date_needlist('JData_inter_product.csv', ['sku_id'])
        result=result.merge(df_pro,on='sku_id',how='inner')
        
        #save the result
        result.to_csv(RESULT, index = False)
        print('end') 